In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

### train 总共31788324个数据，customer 1371980，articles 105542, submit 1371980
### train 里不同customer_id 1362281, 接近10000的customer_id 没有出现
### train 里不同activle_id 104547
### train 里不同顾客在不同日期购买不同商品 数量28575395

In [2]:
train = pd.read_csv('data/transactions_train.csv')
articles=pd.read_csv('data/articles.csv')
customers=pd.read_csv('data/customers.csv')

In [3]:
train['customer_id'] =train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
customers['customer_id'] =customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
train['article_id'] = train['article_id'].astype('int32')
articles['article_id'] = articles['article_id'].astype('int32')

In [4]:
train.t_dat = pd.to_datetime( train.t_dat )
train['year'] = (train.t_dat.dt.year-2000).astype('int8')
train['month'] = (train.t_dat.dt.month).astype('int8')
train['day'] = (train.t_dat.dt.day).astype('int8')

In [5]:
submit=pd.read_csv('data/sample_submission.csv')
submit['customer_id']=submit['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [6]:
def plot_out(df,x):
    sns.set_style("darkgrid")
    f, ax = plt.subplots(figsize=(10,5))
    ax = sns.boxplot(data=df, x=x, color='orange')
    ax.set_xlabel('outliers')
    plt.show()

In [7]:
def look_at_customer(customer_id):
    print('customer_id is ', customer_id)
    print('his purchase num is', len(train[train['customer_id']==customer_id]['customer_id']))
    print('his purchase day num is', train[train['customer_id']==customer_id].t_dat.nunique())
    print('his purchase article num is',train[train['customer_id']==customer_id].article_id.nunique())

In [8]:
## 添加article最早上市时间
article_time_df=train[['t_dat','article_id']].groupby('article_id').min().reset_index().rename(columns={'t_dat':'article_launch_time'}).sort_values('article_launch_time', ascending=False)
articles=articles.merge(article_time_df,how='left',on=['article_id'])
del article_time_df
articles=articles[~np.isnan(articles['article_launch_time'])]

In [9]:
## 添加article上市时间特征
articles['article_launch_year'] = (articles.article_launch_time.dt.year-2000).astype('int8')
articles['article_launch_month'] = (articles.article_launch_time.dt.month).astype('int8')
articles['article_launch_day'] = (articles.article_launch_time.dt.day).astype('int8')
articles['article_launch_bin']=articles['article_launch_month'].apply(lambda x:x//3.5).astype('int8')
del articles['article_launch_time']

In [10]:
## 添加顾客最早购买时间
customer_time_df=train[['t_dat','customer_id']].groupby('customer_id').min().reset_index().rename(columns={'t_dat':'customer_launch_time'})
customers=customers.merge(customer_time_df,how='left',on=['customer_id'])
del customer_time_df
customers=customers[~np.isnan(customers['customer_launch_time'])]
customers['customer_launch_year'] = (customers.customer_launch_time.dt.year-2000).astype('int8')
customers['customer_launch_month'] = (customers.customer_launch_time.dt.month).astype('int8')
customers['customer_launch_day'] = (customers.customer_launch_time.dt.day).astype('int8')

In [11]:
## 添加顾客 总购买数^，购买天数^，购买种类数^，
## 平均购买间隔，最高购买间隔，
## 总购买数^，平均天购买数^，平均种类购买件数^，最高种类购买数，最高天购买数，
## 总购买金额^，平均天购买金额^,平均种类购买金额^,最高种类购买金额=max(种类购买金额），最高天购买金额=max(天购买金额）
agg_dict={'t_dat':['count', 'nunique', 'max'], 'article_id':['nunique'],'price':['sum','mean','max','min','median','std']}
customer_df=train[['customer_id','t_dat','article_id','price']].groupby('customer_id').agg(agg_dict).reset_index()
customer_df.columns=['customer_id','total_purchase_num',
                     'total_purchase_day_num','recent_purchase_day',
                     'purchase_unique_article_num',
                     'total_cost','mean_cost','max_unit_cost','min_unit_cost','median_unit_cost','std_unit_cost']
customer_df['avg_article_purchase_num']=customer_df['total_purchase_num']/customer_df['purchase_unique_article_num']
customer_df['avg_day_purchase_num']=customer_df['total_purchase_num']/customer_df['total_purchase_day_num']
customer_df['avg_day_cost']=customer_df['total_cost']/customer_df['total_purchase_day_num']
customer_df['avg_article_cost']=customer_df['total_cost']/customer_df['purchase_unique_article_num']

In [12]:
customer_day_cnt_df=train.groupby(['customer_id','t_dat'])['price'].count().reset_index().rename(columns={'price':'customer_day_cnt'}).groupby('customer_id')['customer_day_cnt'].max().reset_index().rename(columns={'customer_day_cnt':'customer_max_day_cnt'})
customer_day_cost_df=train.groupby(['customer_id','t_dat'])['price'].sum().reset_index().rename(columns={'price':'customer_day_cost'}).groupby('customer_id')['customer_day_cost'].max().reset_index().rename(columns={'customer_day_cost':'customer_max_day_cost'})
customer_article_cnt_df=train.groupby(['customer_id','article_id'])['price'].count().reset_index().rename(columns={'price':'customer_article_cnt'}).groupby('customer_id')['customer_article_cnt'].max().reset_index().rename(columns={'customer_article_cnt':'customer_max_article_cnt'})
customer_article_cost_df=train.groupby(['customer_id','article_id'])['price'].sum().reset_index().rename(columns={'price':'customer_article_cost'}).groupby('customer_id')['customer_article_cost'].max().reset_index().rename(columns={'customer_article_cost':'customer_max_article_cost'})
customer_df=customer_df.merge(customer_day_cnt_df, how='left', on=['customer_id'])
customer_df=customer_df.merge(customer_day_cost_df, how='left', on=['customer_id'])
customer_df=customer_df.merge(customer_article_cnt_df, how='left', on=['customer_id'])
customer_df=customer_df.merge(customer_article_cost_df, how='left', on=['customer_id'])
del customer_day_cnt_df
del customer_day_cost_df
del customer_article_cnt_df
del customer_article_cost_df

In [13]:
customers=customers.merge(customer_df,how='left',on='customer_id')
del customer_df

In [14]:
customers['day_duration']=(customers['recent_purchase_day']-customers['customer_launch_time']).dt.days

In [15]:
day_diff_df=train[['customer_id','t_dat']].drop_duplicates()

In [16]:
def cal_mean_day_diff(dats):
    dats=dats.values
    if len(dats)==1:
        return None
    dats.sort()
    res=[]
    for i in range(1,len(dats)):
        day0,day1=np.datetime64(dats[i-1]),np.datetime64(dats[i])
        diff=(day1-day0)/np.timedelta64(1, 'D')
        res.append(diff)
    return sum(res)/len(res)

def cal_max_day_diff(dats):
    dats=dats.values
    if len(dats)==1:
        return None
    dats.sort()
    res=[]
    for i in range(1,len(dats)):
        day0,day1=np.datetime64(dats[i-1]),np.datetime64(dats[i])
        diff=(day1-day0)/np.timedelta64(1, 'D')
        res.append(diff)
    return max(res)

In [17]:
mean_day_diff_df=day_diff_df.groupby('customer_id').agg(cal_mean_day_diff).reset_index()
max_day_diff_df=day_diff_df.groupby('customer_id').agg(cal_max_day_diff).reset_index()
mean_day_diff_df.columns=["customer_id","mean_purchase_day_diff"]
max_day_diff_df.columns=["customer_id","max_purchase_day_diff"]

In [18]:
customers=customers.merge(mean_day_diff_df, how='left', on=['customer_id'])
customers=customers.merge(max_day_diff_df, how='left', on=['customer_id'])
del mean_day_diff_df
del max_day_diff_df
del day_diff_df

In [19]:
customer_purchase_cluster_features=['total_purchase_num',
                     'total_purchase_day_num',
                     'purchase_unique_article_num',
                     'total_cost','mean_cost','max_unit_cost','min_unit_cost','median_unit_cost','std_unit_cost',
                     'avg_article_purchase_num','avg_day_purchase_num','avg_day_cost','avg_article_cost',
                      "mean_purchase_day_diff","max_purchase_day_diff"]

In [33]:
## 移除总购买数超过300件顾客数据
remove_customers=list(customers[customers['total_purchase_num']>300].customer_id.values)
customers_1=customers[customers['total_purchase_num']>300]
train_1=train.merge(customers_1['customer_id'],on=['customer_id'])

In [34]:
train_1

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day
0,2018-09-20,1223295072238483821,562252010,0.016932,2,18,9,20
1,2018-09-20,1223295072238483821,574120001,0.027102,2,18,9,20
2,2018-09-20,1223295072238483821,574120001,0.027102,2,18,9,20
3,2018-09-20,1223295072238483821,212042043,0.008458,2,18,9,20
4,2018-09-20,1223295072238483821,590951007,0.008458,2,18,9,20
...,...,...,...,...,...,...,...,...
1325204,2020-07-09,2466263766178831433,554477007,0.010153,2,20,7,9
1325205,2020-07-09,2466263766178831433,554477007,0.010153,2,20,7,9
1325206,2020-07-09,2466263766178831433,554477007,0.010153,2,20,7,9
1325207,2020-07-09,2466263766178831433,514577020,0.006763,2,20,7,9
